# Wstęp do Sztucznej Inteligencji - rok akademicki 2018/2019

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `{NrAlbumu}_{Nazwisko}_{Imie}_{PoprzedniaNazwa}`.

Przed wysłaniem notatnika upewnij się, że rozwiązałeś wszystkie zadania/ćwiczenia, w szczególności, że uzupełniłeś wszystkie pola `YOUR CODE HERE` oraz `YOUR ANSWER HERE`.

## Temat: Algorytmy genetyczne - Lab 4 - Problem plecakowy - Zadania (obowiązkowe)
Zapoznaj się z treścią niniejszego notatnika czytając i wykonując go komórka po komórce. Wykonaj napotkane zadania/ćwiczenia.

### Problem plecakowy
Problem plecakowy to problem optymalizacji dyskretnej. W swojej najprostszej wersji może być sformułowany następująco.

Spośród `N` dostępnych przedmiotów możmy zabrać taką ich liczbę by suma ich wag nie przekraczała pewnej dopuszczalnej wartości (maksymalne obciążenie plecaka). Każdy przedmiot, oprócz wagi, ma przypisaną swoją wartość. 

Które przedmioty zbrać, aby ich sumaryczna wartość była jak największa, ale ich sumaryczna waga nie przekraczała dopuszczalnej maksymalnej wagi plecaka?

Mamy zatem do czynienia z problemem optymalizacji z ograniczeniami.

Przeczytaj więcej o problemie plecakowym:

https://pl.wikipedia.org/wiki/Problem_plecakowy

### Prosty generator problemu plecakowego (0-1 knapsack problem)

Generujemy listę przedmiotów, z losowymi wagami oraz wartościami z podanych przedziałów.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
#%matplotlib notebook

#wmin - minimalna waga przedmiotu
#wmax - maksymalna waga przedmiotu
#vmin - minimalna wartość przedmiotu
#vmax - maksymalna wartość przedmiotu
#items_num - liczba dostępnych przedmiotów
def generate_problem(wmin, wmax, vmin, vmax, items_num):
    w = np.random.randint(wmin, wmax, size=items_num)  #weight
    v = np.random.randint(vmin, vmax, size=items_num)  #values
    return w, v

Przykładowy problem plecakowy (zwróć uwagę, że możemy zagwarantować generowanie za każdym razem tego samego problemu poprzez ustawienie ziarna generatora liczb losowych).

In [2]:
num = 50  # liczba przedmiotów
wmin = 1  # minimlana waga
wmax = 100  # maksymalna waga
vmin = 1  # minimalna wartosc
vmax = 100  # maksymalna wartosc
knapsack_perc = 0.5  # pojemnosc plecaka jako procent sumy wag wszystkich przedmiotow

# ustawienie ziarna
np.random.seed(125701)

w, v = generate_problem(wmin, wmax, vmin, vmax, num)  # w - wagi, v - wartosci
Wall = w.sum()
Vall = v.sum()
W = int(knapsack_perc * Wall) # pojemnosc plecaka

print('Problem plecakowy:')
print('pojemnosc plecaka:', W)
print('wagi:',w, 'suma:', Wall)
print('wartosci:',v, 'suma:', Vall)

Problem plecakowy:
pojemnosc plecaka: 1243
wagi: [35  6 64 18 35 62 47 82 38 27 89 51 94  8 29 14 29 51 77 82 46 36 35  5
 63 35 95 86 38 12  2 54 95 81 93 11 24 56 24 71 68 36 90 50 69 84  3 17
 93 76] suma: 2486
wartosci: [18 83  6  5 38 42 34 93 82 46 74  9 84 80 29 91 37 24 37 79 67 81 78 57
 59 54 99 13 34 90 47 82 65 82 95 25 24 31 45 78 51  4 46 98 41 30 49 77
  8 56] suma: 2657


## Zadanie 1 (1pkt.) - rozwiązanie metodą brute force

Mamy problem plecakowy ze 100 przedmiotami. Chcemy sprawdzić każde możliwe rozwiązanie. Jeśli w ciągu sekundy moglibysmy sprawdzić miliard rozwiązań, to ile lat by to trwało? Przyjmij, że rok ma 365 dni.

Otrzymaną liczbę lat przypisz do zmiennej o nazwie `liczba_lat`. Wynik zaokrąglij do pełnych lat w dół.

In [24]:
# YOUR CODE HERE
n = 100
iteracje = 2**n
liczba_lat = iteracje / 365 / 10**9 / 24 / 60 / 60
print('Zajmie to: {} lat'.format(int(liczba_lat)))

Zajmie to: 40196936841331 lat


In [4]:
#Komórka z autotestem

## Kodowanie rozwiązania

Potencjalne rozwiązanie problemu plecakowego można zakodować jako wektor `0`/`1`, gdzie `1` oznacza wybranie przedmiotu do plecaka.

Warto skorzystać z kodowania `True`/`False` i tablic `numpy`, gdyż ułatwi to obliczenia (macierze True/False mogą służyć do indeksowania innych macierzy).

Przykładowo:

In [5]:
num = w.shape[0]
sol = np.random.randint(0, 2, size=num, dtype=np.bool)  # True oznacza, ze przedmiot jest wybrany

print('Rozwiązanie:', sol)
print('Lista wybranych przedmiotow:', np.arange(num)[sol])
print('Suma wag:', w[sol].sum())
print('Suma wartosci:', v[sol].sum())

Rozwiązanie: [ True False  True False False False  True  True False  True False  True
 False False False False False  True False  True False  True  True False
  True  True  True False  True  True  True False False False  True  True
  True  True  True  True False False False False  True  True  True  True
 False  True]
Lista wybranych przedmiotow: [ 0  2  6  7  9 11 17 19 21 22 24 25 26 28 29 30 34 35 36 37 38 39 44 45
 46 47 49]
Suma wag: 1283
Suma wartosci: 1402


## Rozwiązania dopuszczalne i niedopuszczalne - procedura naprawcza

Jeśli suma wag przekracza pojemność plecaka, to rozwiązanie jest rozwiązaniem niedopuszczalnym i jest całkowicie nieprzydatne.

Rozwiązanie niedopuszczalne można poddać procedurze naprawczej. Przykładowo, poniższa funkcja usuwa przedmioty z plecaka, dopóki nie powstanie rozwiązanie dopuszczalne.

In [6]:
# Usuwa losowe przedmioty, aż rozwiązanie stanie się dopuszczalne
# Jesli rowziazanie jest dopuszczlane, nie zostanie zmienione
def correct_solution(w, v, W, sol):
    num = w.shape[0]
    while w[sol].sum() > W:
        indx = np.random.randint(num)
        while sol[indx%num] == False:
            indx = indx + 1
        sol[indx%num] = False

#### Przykładowa naprawa:

In [7]:
correct_solution(w, v, W, sol)

print('Rozwiązanie:', sol)
print('Lista wybranych przedmiotow:', np.arange(num)[sol])
print('Suma wag:', w[sol].sum())
print('Suma wartosci:', v[sol].sum())

Rozwiązanie: [ True False  True False False False  True  True False  True False  True
 False False False False False  True False  True False  True  True False
  True  True  True False  True  True  True False False False  True  True
  True  True  True False False False False False  True  True  True  True
 False  True]
Lista wybranych przedmiotow: [ 0  2  6  7  9 11 17 19 21 22 24 25 26 28 29 30 34 35 36 37 38 44 45 46
 47 49]
Suma wag: 1212
Suma wartosci: 1324


## Losowe, dopuszczalne rozwiązania

Bazując na powyższej procedurze, można zdefiniować funkcję generującą losowe, ale zawsze dopuszczalne rozwiązania.

In [8]:
def get_random_solution(w, v, W):
    num = w.shape[0]
    sol = np.random.randint(0,2, size=num, dtype=np.bool)  # 1 / True oznacza, ze przedmiot jest wybrany
    _V = np.sum(v[sol])
    _W = np.sum(w[sol])
    if _W > W:
        correct_solution(w,v,W,sol)
        _V = np.sum(v[sol])
        _W = np.sum(w[sol])
    return sol, _W, _V

#### Przykład użycia:

In [9]:
sol = get_random_solution(w,v,W)

print('Rozwiązanie:', sol[0])
print('Lista wybranych przedmiotow:', np.arange(num)[sol[0]])
print('Suma wag:', sol[1])
print('Suma wartosci:', sol[2])

Rozwiązanie: [ True  True False  True  True False False False False False  True  True
  True  True False False  True False  True False  True  True  True False
 False  True  True False False  True False False False False False  True
 False False False False False  True False False False False False False
 False False]
Lista wybranych przedmiotow: [ 0  1  3  4 10 11 12 13 16 18 20 21 22 25 26 29 35 41]
Suma wag: 748
Suma wartosci: 963


## Random search

Powyższe funkcje można przełożyć na prostą heurystykę przeszukiwania losowego. Generujemy losowe (ale dopuszczalne) rozwiązania przez zadaną liczbę iteracji i zapamiętujemy najlepsze. Dodatkowo, zapisujemy oceny rozwiązań (wartość wszystkich przedmiotów w plecaku) by przedstawić je na wykresie.

In [10]:
def search_random(w,v,W,iters):
    best_sol, best_W, best_V = get_random_solution(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    for i in range(iters):
        sol, _W, _V = get_random_solution(w,v,W)
        if best_V < _V:
            best_sol, best_W, best_V = sol, _W, _V
        v_all.append(_V)
        v_best.append(best_V)
    #plt.figure()
    #plt.plot(v_all)
    #plt.plot(v_best)
    #plt.show()
    return best_sol, best_W, best_V, v_all, v_best

#### Przykładowe uruchomienie:

In [11]:
sol_random_search = search_random(w, v, W, 1000)

print('Najlepsze rozwiązanie:',sol_random_search[0])
print('Przedmioty:',np.arange(num)[sol_random_search[0]])
print('Suma wag:', sol_random_search[1])
print('Suma wartosci:', sol_random_search[2])

Najlepsze rozwiązanie: [ True  True False False  True False False  True False False  True  True
  True  True False  True False False False  True  True  True  True  True
 False False False False False  True False  True  True  True  True  True
  True  True  True False False False False  True False False False  True
 False  True]
Przedmioty: [ 0  1  4  7 10 11 12 13 15 19 20 21 22 23 29 31 32 33 34 35 36 37 38 43
 47 49]
Suma wag: 1211
Suma wartosci: 1702


## Greedy search - procedura optymalizacji zachłannej

Problemem w powyższym podejściu jest fakt, że kolejne rozwiązania nie korzystają z uzyskanej już wiedzy o najlepszych do tej pory rozwiązaniach. Zatem inna prosta heurytyka polega na wystartowaniu z losowego rozwiązania, a następnie modyfikowaniu go poprzez losowe dodawanie przedmiotu do plecaka. Jeśli zmiana (po ewentualnej naprawie) wprowadza poprawę, pozostajemy przy takim rozwiązaniu, jeśli następuje pogorszenie, odrzucamy takie rozwiązanie i ponawiamy próbę.

Taka procedura jest przykładem optymalizacji zachłannej.

In [12]:
def search_greedy_improvement(w, v, W, iters):
    best_sol, best_W, best_V = get_random_solution(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    num = w.shape[0]
    for i in range(iters):
        sol = best_sol.copy()
        #set random 0 bit to 1
        indx = np.random.randint(num)
        while sol[indx%num] == True:
            indx = indx + 1
        sol[indx%num] = True
        #correct if needed
        if w[sol].sum() > W:
            correct_solution(w,v,W,sol)
        _V = v[sol].sum()
        _W = w[sol].sum()
        if best_V < _V:
            best_sol, best_W, best_V = sol.copy(), _W, _V
        v_all.append(_V)
        v_best.append(best_V)
    #plt.figure()
    #plt.plot(v_all)
    #plt.plot(v_best)
    #plt.show()
    return best_sol, best_W, best_V, v_all, v_best 

#### Przykładowe uruchomienie:

In [13]:
sol_greedy = search_greedy_improvement(w,v,W,1000)

print('Najlepsze rozwiązanie:',sol_greedy[0])
print('Przedmioty:',np.arange(num)[sol_greedy[0]])
print('Suma wag:', sol_greedy[1])
print('Suma wartosci:', sol_greedy[2])

Najlepsze rozwiązanie: [False  True False False False  True False  True  True  True False False
  True  True  True  True  True False False  True  True  True  True  True
  True  True  True False False  True  True  True False  True  True  True
  True False  True  True False False False  True False False  True  True
 False False]
Przedmioty: [ 1  5  7  8  9 12 13 14 15 16 19 20 21 22 23 24 25 26 29 30 31 33 34 35
 36 38 39 43 46 47]
Suma wag: 1228
Suma wartosci: 2033


## Dwie inne proste heurystyki

### Najpierw najbardziej wartościowe

Jak dobre rozwiązanie dostaniemy, jeśli do plecaka pakować będziemy najpierw najbardziej wartościowe przedmioty (o ile się zmieszczą)? Pomysł ten jest zaimplementowany w poniższej funkcji.


In [14]:
#Pakuje najpierw najbardziej wartościowe przedmioty
def get_value_first(w, v, W):
    ii = np.argsort(-v)
    num = w.shape[0]
    sol = np.repeat(False, num)
    _W = 0
    for i in range(num):
        if _W + w[ii[i]] <= W:
            sol[ii[i]] = True
            _W = _W + w[ii[i]]
    _V = v[sol].sum()
    return sol, _W, _V

#### Przykładowe uruchomienie:

In [15]:
sol_value_first = get_value_first(w, v, W)

print('Najlepsze rozwiązanie:',sol_value_first[0])
print('Przedmioty:',np.arange(num)[sol_value_first[0]])
print('Suma wag:', sol_value_first[1])
print('Suma wartosci:', sol_value_first[2])

Najlepsze rozwiązanie: [False  True False False False False False  True  True False  True False
  True  True False  True False False False  True  True  True  True  True
  True False  True False False  True False  True  True  True  True False
 False False False  True False False False  True False False False  True
 False  True]
Przedmioty: [ 1  7  8 10 12 13 15 19 20 21 22 23 24 26 29 31 32 33 34 39 43 47 49]
Suma wag: 1242
Suma wartosci: 1830


### Najpierw te o najlepszym stosunku wartości do wagi

Inny pomysł to pakowanie najpierw przedmiotów o najlepszym stosunku wartości do wagi.


In [16]:
def get_ratio_first(w, v, W):
    ii = np.argsort(-v/w) #stosunek wartosci do wagi
    num = w.shape[0]
    sol = np.repeat(False, num)
    _W = 0
    for i in range(num):
        if _W + w[ii[i]] <= W:
            sol[ii[i]] = True
            _W = _W + w[ii[i]]
    _V = v[sol].sum()
    return sol, _W, _V

#### Przykładowe uruchomienie:

In [17]:
sol_ratio_first = get_ratio_first(w, v, W)

print('Najlepsze rozwiązanie:',sol_ratio_first[0])
print('Przedmioty:',np.arange(num)[sol_ratio_first[0]])
print('Suma wag:', sol_ratio_first[1])
print('Suma wartosci:', sol_ratio_first[2])

Najlepsze rozwiązanie: [False  True False False  True False False  True  True  True False False
  True  True  True  True  True False False  True  True  True  True  True
  True  True  True False  True  True  True  True False  True  True  True
  True False  True  True False False False  True False False  True  True
 False False]
Przedmioty: [ 1  4  7  8  9 12 13 14 15 16 19 20 21 22 23 24 25 26 28 29 30 31 33 34
 35 36 38 39 43 46 47]
Suma wag: 1239
Suma wartosci: 2063


## Zadanie 2 (3 pkt.)

Opracowane być mogą inne procedury naprawcze. Przykładowo, dla rozwiązania niedopuszczalengo, zamiast zmieniać losowe bity True na False, jak w funkcji `correct_solution`, można usuwać najpierw te przedmioty, które mają najgorszy stosunek wartości do wagi. Zaimplementuj taką procedurę. 

Porównaj jej działanie z funkcją `correct_solution` w metodach RandomSearch oraz GreedySearch. Przedstaw uśrednione wyniki (co najmniej 10 uruchomień) i wnioski.

In [26]:

def correct_solution2(w, v, W, sol):
    narr = v/w
    
    num = w.shape[0]
    while w[sol].sum() > W:
        result = np.where(narr == np.amin(narr[sol]))
        indx = result[0]
        #print(narr)
        #print(indx)
        sol[indx] = False
        
def get_random_solution_v2(w, v, W):
    num = w.shape[0]
    sol = np.random.randint(0,2, size=num, dtype=np.bool)  # 1 / True oznacza, ze przedmiot jest wybrany
    _V = np.sum(v[sol])
    _W = np.sum(w[sol])
    if _W > W:
        correct_solution2(w,v,W,sol)
        _V = np.sum(v[sol])
        _W = np.sum(w[sol])
    return sol, _W, _V

def search_greedy_improvement_v2(w, v, W, iters):
    best_sol, best_W, best_V = get_random_solution_v2(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    num = w.shape[0]
    for i in range(iters):
        sol = best_sol.copy()
        #set random 0 bit to 1
        indx = np.random.randint(num)
        while sol[indx%num] == True:
            indx = indx + 1
        sol[indx%num] = True
        #correct if needed
        if w[sol].sum() > W:
            correct_solution2(w,v,W,sol)
        _V = v[sol].sum()
        _W = w[sol].sum()
        if best_V < _V:
            best_sol, best_W, best_V = sol.copy(), _W, _V
        v_all.append(_V)
        v_best.append(best_V)

    return best_sol, best_W, best_V, v_all, v_best 


def search_random_v2(w,v,W,iters):
    best_sol, best_W, best_V = get_random_solution_v2(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    for i in range(iters):
        sol, _W, _V = get_random_solution_v2(w,v,W)
        if best_V < _V:
            best_sol, best_W, best_V = sol, _W, _V
        v_all.append(_V)
        v_best.append(best_V)
    
    return best_sol, best_W, best_V, v_all, v_best


w, v = generate_problem(wmin, wmax, vmin, vmax, num)  # w - wagi, v - wartosci
Wall = w.sum()
Vall = v.sum()
W = int(knapsack_perc * Wall) # pojemnosc plecaka
num = w.shape[0]

##############
#ILOSC TESTOW#
##############
tests = 20
##############

sol_greedys,sol_random_searchs,sol_greedys2,sol_random_searchs2 = np.ndarray(shape=(tests,2)),np.ndarray(shape=(tests,2)),np.ndarray(shape=(tests,2)),np.ndarray(shape=(tests,2))
for i in range(tests):
    g = search_greedy_improvement(w,v,W,1000)
    rs = search_random(w, v, W, 1000)
    sol_greedys[i] = [g[1], g[2]]
    sol_random_searchs[i] = [rs[1], rs[2]]
    #v2
    g = search_greedy_improvement_v2(w,v,W,1000)
    rs = search_random_v2(w, v, W, 1000)
    sol_greedys2[i] = [g[1], g[2]]
    sol_random_searchs2[i] = [rs[1], rs[2]]
    
print("Domyslna metoda search_greedy:")
print('>Suma wag:', np.sum(a=sol_greedys, axis=0)[0]/tests)
print('>Suma wartosci:', np.sum(a=sol_greedys, axis=0)[1]/tests)
print("Domyslna metoda search_random:")
print('>Suma wag:', np.sum(a=sol_random_searchs, axis=0)[0]/tests)
print('>Suma wartosci:', np.sum(a=sol_random_searchs, axis=0)[1]/tests)
print("\nWlasna metoda search_greedy:")
print('>Suma wag:', np.sum(a=sol_greedys2, axis=0)[0]/tests)
print('>Suma wartosci:', np.sum(a=sol_greedys2, axis=0)[1]/tests)
print("Wlasna metoda search_random:")
print('>Suma wag:', np.sum(a=sol_random_searchs2, axis=0)[0]/tests)
print('>Suma wartosci:', np.sum(a=sol_random_searchs2, axis=0)[1]/tests)


Domyslna metoda search_greedy:
>Suma wag: 1251.65
>Suma wartosci: 1986.6
Domyslna metoda search_random:
>Suma wag: 1218.3
>Suma wartosci: 1603.35

Wlasna metoda search_greedy:
>Suma wag: 1257.55
>Suma wartosci: 2022.15
Wlasna metoda search_random:
>Suma wag: 1225.95
>Suma wartosci: 1754.2


Twoje wyniki i wnioski umieść w komórce poniżej.

Wnioski:
1. W metodzie GreedySearch nie jest zauważalna duza roznica. Wynika to glownie z faktu poprawiania obecnego wyniku.
2. Z kolei w metodzie RandomSearch widzimy za to znaczna poprawe przekraczajaca 10%. Tutaj głównie owa optymalizacja sie przydaje ze wzgledu ciaglej losowosci.

In [19]:
#Komórka z autotestem

## Zadanie 3 (6 pkt.)

Dostosuj swoją implementację algorytmu genetycznego do problemu plecakowego. 

- Jakie wyniki można uzyskać z jego pomocą? Czy działa on zawsze lepiej niż inne heurystyki?

- Która procedura naprawcza działa lepiej w algorytmie genetycznym?

- Przedstaw wnioski na podstawie uśrednionych wyników dla problemów plecakowych o rozmiarze 50, 100, 300.

- Problemy plecakowe i najlepsze znalezione rozwiązania zapisz do plików.

UWAGA! Po wygenerowaniu problemu plecakowego, przed uruchomieniem algorytmu genetycznego (lub innego) wywołaj:

`np.random.seed(int(time.time()))`

tak by problem plecakowy generował się ten sam, ale algorytmy miały szansę na nowy przebieg.

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib notebook

class modif():
    mode = 1
    
def correct_solution1(w, v, W, sol):
    #print("1")
    num = w.shape[0]
    while w[sol].sum() > W:
        indx = np.random.randint(num)
        while sol[indx%num] == False:
            indx = indx + 1
        sol[indx%num] = False
    return sol


def correct_solution2(w, v, W, sol):
    #print("2")
    narr = v / w
    num = w.shape[0]
    while w[sol].sum() > W:
        result = np.where(narr == np.amin(narr[sol]))
        indx = result[0]
        
        sol[indx] = False
    return sol


def generate_problem(wmin, wmax, vmin, vmax, items_num):
    w = np.random.randint(wmin, wmax, size=items_num)  #weight
    v = np.random.randint(vmin, vmax, size=items_num)  #values
    return w, v


def gen_pop(w, v, W, pop_size):
    pop = np.random.randint(0, 2, size=(pop_size, len(w)), dtype=np.bool)
    for p in pop:
        if modif.mode==1:
            #print("Metoda naprawcza 1")
            p = correct_solution1(w, v, W, p)
        else:
            #print("Metoda naprawcza 2")
            p = correct_solution2(w, v, W, p)
    return pop

def evaluate(pop, v):
    evaluated_pop = np.zeros(shape=len(pop), dtype="float")
    for i in range(len(pop)):
        evaluated_pop[i] = np.sum(v[pop[i]])
    #print(evaluated_pop)
    return evaluated_pop

#Selekcja
def select(pop, evals):
    sum = np.sum(evals)
    new_pop = np.copy(pop)
    roulet = np.zeros(shape=len(evals), dtype="float")
    for i in range(len(evals)):
        roulet[i] = roulet[i-1] + (evals[i] / sum)
    for i in range(len(pop)):
        rand = np.random.rand()
        for j in range(len(evals)):
            if evals[j] > rand:
                new_pop[i] = pop[j]
                break

    return new_pop


#Krzyzowanie
def xover(pop, pc, w, v, W):  # pc - prawdopodobienstwo krzyzowania
    new_pop = np.copy(pop)
    for i in range(0, len(pop)-1, 2):
        if np.random.rand() <= pc:
            point = int(np.random.rand() * (len(pop[i]) - 1))
            narr1 = np.concatenate((pop[i][0:point], pop[i+1][point:]))
            narr2 = np.concatenate((pop[i+1][0:point], pop[i][point:]))
            new_pop[i] = np.array(narr1)
            new_pop[i+1] = np.array(narr2)
            
    #Wybor implementacji metody poprawczej
    for p in new_pop:
        if modif.mode==1:
            p = correct_solution1(w, v, W, p)
        else:
            p = correct_solution2(w, v, W, p)

    return new_pop


#Mutacja
def mutate(pop, pm, w, v, W):  # pm - prawdopodobienstwo mutacji, pop = rozwiazanie
    
    for i in range(len(pop)):
        for j in range(len(pop[i])):
            if np.random.rand() <= pm:
                if pop[i][j] == True:
                    pop[i][j] = False
                else:
                    pop[i][j] = True
                    
        #Wybor implementacji metody poprawczej
        if modif.mode==1:
            pop[i] = correct_solution1(w, v, W, pop[i])
        else:
            pop[i] = correct_solution2(w, v, W, pop[i])
            
    return pop


def evolve_knapsack(w, v, W, pop_size, pxover, pmutate, generations):
    pop = gen_pop(w, v, W, pop_size)
    evals = evaluate(pop, v)
    i = np.argmax(evals)
    best = pop[i].copy()
    best_V = evals[i]
    best_iter = 0
    v_all = [best_V]
    v_best = [best_V]
    v_mean = [np.mean(evals)]

    for i in range(generations):
        pop = select(pop, evals)
        pop = xover(pop, pxover, w, v, W)
        pop = mutate(pop, pmutate, w, v, W)
        evals = evaluate(pop, v)
        ii = np.argmax(evals)
        temp_best_v = evals[ii]
        if temp_best_v > best_V:
            best_V = temp_best_v
            best_iter = i + 1
            best = pop[ii].copy()
        v_all.append(temp_best_v)
        v_best.append(best_V)
        v_mean.append(np.mean(evals))

    #plt.plot(v_best, 'r')
    #plt.plot(v_all, 'g')
    #plt.plot(v_mean, 'b')
    #plt.show()
    return best, w[best].sum(), best_V


def test(corr_sol, tests, w, v, W, pop_size, pxover, pmutate, generations):
    cs = corr_sol
    wbests = np.zeros(shape=tests, dtype="float")
    vbests = np.zeros(shape=tests, dtype="float")
    pbests = np.zeros(shape=(tests, len(w)) , dtype="bool")
    for t in range(tests):
        best, best_w, best_v = evolve_knapsack(w, v, W, pop_size, pxover, pmutate, generations)
        
        pbests[t] = best
        wbests[t] = best_w
        vbests[t] = best_v
    
    ##Najlepszy osobnik
    result = np.argmax(vbests)
    best_pop = pbests[result]
    return np.average(wbests), np.average(vbests), best_pop

def gen_multi_test(rozmiar_problemu):
    num = rozmiar_problemu  # liczba przedmiotów
    wmin = 1  # minimlana waga
    wmax = 100  # maksymalna waga
    vmin = 1  # minimalna wartosc
    vmax = 100  # maksymalna wartosc
    knapsack_perc = 0.5  # pojemnosc plecaka jako procent sumy wag wszystkich przedmiotow


    #ustawienie ziarna
    np.random.seed(125701)

    w, v = generate_problem(wmin, wmax, vmin, vmax, num)  # w - wagi, v - wartosci
    Wall = w.sum()
    Vall = v.sum()
    W = int(knapsack_perc * Wall) # pojemnosc plecaka
    print("Problem plecakowy rozmiaru", num)
    print('Problem plecakowy:')
    print('pojemnosc plecaka:', W)
    print('wagi:',w, 'suma:', Wall)
    print('wartosci:',v, 'suma:', Vall)

    np.random.seed(int(time.time()))

    pop_size = 50
    pxover = 0.9
    pmutate = 0.02
    generations = 100
    tests = 20
    
    
    
    #Test 1
    modif.mode = 1
    w1, v1, bpop1 = test(1, tests, w, v, W, pop_size, pxover, pmutate, generations)
    print("Algorytm genetyczny korekcji losowej:")
    np.savetxt('Algorytm_genetyczny_korekcji_losowej_rozmiaru' + str(num) + ".out", bpop1, delimiter=',')
    print("Zapisuje najwyzszy wynik do pliku")
    print(">Srednia waga:", w1)
    print(">Srednia wartosc:", v1)

    #Test 2
    modif.mode = 2
    w2, v2, bpop2 = test(2, tests, w, v, W, pop_size, pxover, pmutate, generations)
    print("Algorytm genetyczny korekcji napisanej przez nas:")
    np.savetxt('Algorytm_genetyczny_korekcji_losowej_own_rozmiaru' + str(num) + ".out", bpop2, delimiter=',')
    print("Zapisuje najwyzszy wynik do pliku")
    print(">Srednia waga:", w2)
    print(">Srednia wartosc:", v2)

    
def heu_multi_test(rozmiar_problemu):
    num = rozmiar_problemu  # liczba przedmiotów
    wmin = 1  # minimlana waga
    wmax = 100  # maksymalna waga
    vmin = 1  # minimalna wartosc
    vmax = 100  # maksymalna wartosc
    knapsack_perc = 0.5  # pojemnosc plecaka jako procent sumy wag wszystkich przedmiotow


    #ustawienie ziarna
    np.random.seed(125701)

    w, v = generate_problem(wmin, wmax, vmin, vmax, num)  # w - wagi, v - wartosci
    Wall = w.sum()
    Vall = v.sum()
    W = int(knapsack_perc * Wall)
    
    tests = 20

    np.random.seed(int(time.time()))    
    
    #Test 1
    print("Rozwiazanie heurystyka random_search:")
    wbests = np.zeros(shape=tests, dtype="float")
    vbests = np.zeros(shape=tests, dtype="float")
    for i in range(tests):
        sol_random_search = search_random(w, v, W, 1000)
        wbests[i] = sol_random_search[1]
        vbests[i] = sol_random_search[2]
    
    print('>Srednia waga:', np.average(wbests))
    print('>Srednia wartosc:', np.average(vbests))

    #Test 2
    print("Rozwiazanie heurystyka greedy_search:")
    wbests = np.zeros(shape=tests, dtype="float")
    vbests = np.zeros(shape=tests, dtype="float")
    for i in range(tests):
        sol_greedy = search_greedy_improvement(w, v, W, 1000)
        wbests[i] = sol_greedy[1]
        vbests[i] = sol_greedy[2]
    
    print('>Srednia waga:', np.average(wbests))
    print('>Srednia wartosc:', np.average(vbests))
    
    #Test 3
    print("Rozwiazanie heurystyka value_first:")
    wbests = np.zeros(shape=tests, dtype="float")
    vbests = np.zeros(shape=tests, dtype="float")
    for i in range(tests):
        sol_value_first = get_value_first(w, v, W)
        wbests[i] = sol_value_first[1]
        vbests[i] = sol_value_first[2]
    
    print('>Srednia waga:', np.average(wbests))
    print('>Srednia wartosc:', np.average(vbests))
    
    #Test 4
    print("Rozwiazanie heurystyka ratio_first:")
    wbests = np.zeros(shape=tests, dtype="float")
    vbests = np.zeros(shape=tests, dtype="float")
    for i in range(tests):
        sol_ratio_first = get_ratio_first(w, v, W)
        wbests[i] = sol_ratio_first[1]
        vbests[i] = sol_ratio_first[2]
    
    print('>Srednia waga:', np.average(wbests))
    print('>Srednia wartosc:', np.average(vbests))
    print("\n")
    
    
#zmienna globalna
cs=0
gen_multi_test(50)
heu_multi_test(50)
gen_multi_test(100)
heu_multi_test(100)
gen_multi_test(300)
heu_multi_test(300)

Problem plecakowy rozmiaru 50
Problem plecakowy:
pojemnosc plecaka: 1243
wagi: [35  6 64 18 35 62 47 82 38 27 89 51 94  8 29 14 29 51 77 82 46 36 35  5
 63 35 95 86 38 12  2 54 95 81 93 11 24 56 24 71 68 36 90 50 69 84  3 17
 93 76] suma: 2486
wartosci: [18 83  6  5 38 42 34 93 82 46 74  9 84 80 29 91 37 24 37 79 67 81 78 57
 59 54 99 13 34 90 47 82 65 82 95 25 24 31 45 78 51  4 46 98 41 30 49 77
  8 56] suma: 2657
Algorytm genetyczny korekcji losowej:
Zapisuje najwyzszy wynik do pliku
>Srednia waga: 1206.5
>Srednia wartosc: 1581.75
Algorytm genetyczny korekcji napisanej przez nas:
Zapisuje najwyzszy wynik do pliku
>Srednia waga: 1213.05
>Srednia wartosc: 1767.8
Rozwiazanie heurystyka random_search:
>Srednia waga: 1220.9
>Srednia wartosc: 1688.05
Rozwiazanie heurystyka greedy_search:
>Srednia waga: 1237.35
>Srednia wartosc: 2033.8
Rozwiazanie heurystyka value_first:
>Srednia waga: 1242.0
>Srednia wartosc: 1830.0
Rozwiazanie heurystyka ratio_first:
>Srednia waga: 1239.0
>Srednia wartosc

Twoje wyniki i wnioski umieść w komórce poniżej.

Testy wykonane dla 1000 pokolen w algorytmach genetycznych oraz 1000 iteracji w metodach heurystycznych. Ilość osobników nie wpływała znacznie na wyniki testów.
***
Jakie wyniki można uzyskać z jego pomocą? Czy działa on zawsze lepiej niż inne heurystyki?
>Z powyzszych wynikow uzyskujemy wnioski:
1. Algorytm genetyczny działa lepiej tylko od metody przeszukiwania losowego, pozostałe heurystyki osiagaja wyższe rezultaty.
2. Najlepsza okazuje sie heurystyka ratio_first
3. Algorytm genetyczny jest wyraźnie wolniejszą metodą
***
Która procedura naprawcza działa lepiej w algorytmie genetycznym?
1. Dla wielokrotnego uruchomienia testu metoda ktora tworzylismy jest znacznie lepsza.
2. Wraz ze wzrostem rozmiaru plecaka różnica rośnie tylko nieznacznie.
***
Przedstaw wnioski na podstawie uśrednionych wyników dla problemów plecakowych o rozmiarze 50, 100, 300:
1. Przewaga najlepszej heurystyki wzgledem algorytmu genetycznego rosnie wraz pojemnoscia plecaka. Dla rozmiaru 50 wynosi ona 10% a dla rozmiaru 300 juz prawie 25%.
2. Wykorzystenie heurystyk jest w przypadku tego zadanie lepszym pomysłem. Działanie algorytmu osiaga zblizone wyniki do najgorszej heurystyki, pozostałe osiagaja duzo lepsze wyniki.
3. Czas wykonywania sie algorytmu dla wiekszych problemow genetycznych trwałby zbyt długo, juz dla problemu=300 trwało to kilkadziesiat sekund. Heurystyki tymczasem wykonały to samo zadanie ponizej jednej sekundy.
***
Wyniki:
1. Problem plecakowy rozmiaru 50, pojemnosc plecaka 1243:
    1. Algorytm genetyczny korekcji losowej:
        1. Srednia waga: 1214.9
        2. Srednia wartosc: 1795.9
    2. Algorytm genetyczny korekcji napisanej przez nas:
        1. Srednia waga: 1217.4
        2. Srednia wartosc: 1807.9
    3. Rozwiazanie heurystyka random_search:
        1. Srednia waga: 1222.6
        2. Srednia wartosc: 1677.1
    4. Rozwiazanie heurystyka greedy_search:
        1. Srednia waga: 1237.15
        2. Srednia wartosc: 2021.8
    5. Rozwiazanie heurystyka value_first:
        1. Srednia waga: 1242.0
        2. Srednia wartosc: 1830.0
    6. Rozwiazanie heurystyka ratio_first:
        1. Srednia waga: 1239.0
        2. Srednia wartosc: 2063.0

2. Problem plecakowy rozmiaru 100, pojemnosc plecaka 2571:
    1. Algorytm genetyczny korekcji losowej:
        1. Srednia waga: 2524.9
        2. Srednia wartosc: 3362.2
    2. Algorytm genetyczny korekcji napisanej przez nas:
        1. Srednia waga: 2533.7
        2. Srednia wartosc: 3378.35
    3. Rozwiazanie heurystyka random_search:
        1. Srednia waga: 2537.9
        2. Srednia wartosc: 3182.4
    4. Rozwiazanie heurystyka greedy_search:
        1. Srednia waga: 2564.4
        2. Srednia wartosc: 4119.3
    5. Rozwiazanie heurystyka value_first:
        1. Srednia waga: 2571.0
        2. Srednia wartosc: 4056.0
    6. Rozwiazanie heurystyka ratio_first:
        1. Srednia waga: 2568.0
        2. Srednia wartosc: 4240.0

3. Problem plecakowy rozmiaru 300, pojemnosc plecaka 7685:
    1. Algorytm genetyczny korekcji losowej:
        1. Srednia waga: 7654.5
        2. Srednia wartosc: 8300.6
    2. Algorytm genetyczny korekcji napisanej przez nas:
        1. Srednia waga: 7641.0
        2. Srednia wartosc: 8256.55
    3. Rozwiazanie heurystyka random_search:
        1. Srednia waga: 7619.45
        2. Srednia wartosc: 7895.95
    4. Rozwiazanie heurystyka greedy_search:
        1. Srednia waga: 7679.15
        2. Srednia wartosc: 10397.9
    5. Rozwiazanie heurystyka value_first:
        1. Srednia waga: 7685.0
        2. Srednia wartosc: 10670.0
    6. Rozwiazanie heurystyka ratio_first:
        1. Srednia waga: 7684.0
        2. Srednia wartosc: 11227.0

<div style="text-align: right">&copy; Zakład Inteligencji Obliczeniowej, Instytut Informatyki, Politechnika Krakowska </div>